In [1]:
from pyspark.sql.functions import col, sum, acos, cos, radians, sin
from pyspark.sql.types import FloatType
from pyspark.sql.types import *
import pyspark 
from delta import *
from delta.tables import DeltaTable

def main():

    # Constants
    DISTANCE = 2 # km
    TABLE_PATH = "hdfs:///project/data/business_data/delta_table_proximity_count"
    CSV_PATH = "hdfs:///project/data/business_data/yelp_academic_dataset_business.csv"
    
    # Initiate spark
    builder = pyspark.sql.SparkSession.builder.appName("spark_proximity_count_no_udfs") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.executor.cores", 4)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    
    # Create dataframe from csv file
    schema = "Business_id STRING, Latitude FLOAT, Longitude FLOAT, Stars FLOAT"
    df = spark.read.csv(CSV_PATH, schema=schema).repartition(200, "Business_id")

    # Combination of all business pairs
    # df_pairs = df.toDF("Business_id1", "Latitude1", "Longitude1", "Stars1").crossJoin(df.toDF("Business_id2", "Latitude2", "Longitude2", "Stars2"))

    df.createOrReplaceTempView("df1")
    df.createOrReplaceTempView("df2")
    df_pairs = spark.sql(
        """
        SELECT 
        df1.Business_id AS Business_id1
        ,df2.Business_id AS Business_id2
        ,df1.Latitude AS Latitude1
        ,df1.Longitude AS Longitude1
        ,df2.Latitude AS Latitude2
        ,df2.Longitude AS Longitude2
        FROM df1, df2
        WHERE df1.Business_id <> df2.Business_id
        """
    )

    # Calculate haversine distances
    df_pairs = df_pairs.withColumn("WithinProximity", 
        ((acos(sin(radians("Latitude1")) * sin(radians("Latitude2")) +
            cos(radians("Latitude1")) * cos(radians("Latitude2")) *
            cos(radians("Longitude2") - radians("Longitude1"))) * 6371
    ) <= DISTANCE).cast("integer"))
    df_proximity = df_pairs.groupBy(col("Business_id1")).agg(sum("WithinProximity").alias("ProximityCount"))
    df = df.join(df_proximity, on=df.Business_id == df_proximity.Business_id1, how="inner").drop("Business_id1")

    # If no delta table exists, save and exit
    if not DeltaTable.isDeltaTable(spark, TABLE_PATH):
        df.write.format("delta").save(TABLE_PATH)
        return
        
    # # Upsert delta table
    delta_table = DeltaTable.forPath(spark, TABLE_PATH)
    delta_table.alias("old") \
        .merge(
            df.alias("new"),
            "old.Business_id = new.Business_id"
        ) \
        .whenMatchedUpdate(set=
            {
                "Business_id": "new.Business_id",
                "Latitude": "new.Latitude",
                "Longitude": "new.Longitude",
                "Stars": "new.Stars",
                "ProximityCount": "new.ProximityCount"
            }
        ) \
        .whenNotMatchedInsert(values=
            {
                "Business_id": "new.Business_id",
                "Latitude": "new.Latitude",
                "Longitude": "new.Longitude",
                "Stars": "new.Stars",
                "ProximityCount": "new.ProximityCount"
            }
        ) \
        .execute()


if __name__ == "__main__":
    main()

:: loading settings :: url = jar:file:/home/ubuntu/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7401e71d-ce0b-4766-a9ca-bdeb0fa81608;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 1302ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

23/04/26 20:27:37 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/04/26 20:28:03 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar added multiple times to distributed cache.
23/04/26 20:28:03 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-storage-2.3.0.jar added multiple times to distributed cache.
23/04/26 20:28:03 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar added multiple times to distributed cache.


+--------------------+---------+-----------+-----+--------------+
|         Business_id| Latitude|  Longitude|Stars|ProximityCount|
+--------------------+---------+-----------+-----+--------------+
|--ZVrH2X2QXBFdCil...|  39.9973| -75.292206|  4.5|            30|
|--_9CAxgfXZmoFdNI...|27.772844| -82.676155|  3.5|            20|
|--hF_3v1JmU9nlu4z...|39.767887|  -86.15835|  4.5|            82|
|-02xFuruu85XmDn2x...|32.251038|-110.833176|  4.5|            21|
|-0Ym1Wg3bXd_TDz8J...|53.518486| -113.49318|  4.5|            41|
|-0fvhILrC9UsQ6gLN...|40.046192|  -75.01509|  4.5|            13|
|-1ueCbvIpUPi8KT95...| 39.50442| -119.77959|  4.0|            51|
|-2Axhv9AZ_n7qjQef...|36.216606|  -86.59637|  3.5|             2|
|-2bLM6nIETD4Mk-rh...|39.768684|  -86.15352|  5.0|            78|
|-2ke_JDOpgTZWqirM...| 39.71531|  -86.08123|  3.5|             4|
|-3-6BB10tIWNKGEF0...| 32.28854| -110.96314|  4.0|            30|
|-3AooxIkg38UyUdlz...|34.421932| -119.70219|  3.0|           172|
|-3YTjk_J0